In [0]:
-- Set context
USE CATALOG workspace;
USE SCHEMA gold;


In [0]:
-- VIEW 1: Customer Dimension
CREATE OR REPLACE VIEW dim_customers AS
SELECT
  ROW_NUMBER() OVER (ORDER BY ci.cst_id) AS customer_key, -- surrogate key
  ci.cst_id AS customer_id,
  ci.cst_key AS customer_number,
  ci.cst_firstname AS first_name,
  ci.cst_lastname AS last_name,
  la.cntry AS country,
  ci.cst_marital_status AS marital_status,
  CASE 
    WHEN ci.cst_gndr != 'n/a' THEN ci.cst_gndr 
    ELSE COALESCE(ca.gen, 'n/a') 
  END AS gender,
  ca.bdate AS birthdate,
  ci.cst_create_date AS create_date
FROM silver_crm.silver_crm_cust_info ci
LEFT JOIN silver_erp.silver_erp_cust_az12 ca
  ON TRIM(UPPER(ci.cst_key)) = TRIM(UPPER(ca.cid))
LEFT JOIN silver_erp.silver_erp_loc_a101 la
  ON TRIM(UPPER(ci.cst_key)) = TRIM(UPPER(la.cid));

In [0]:
SELECT * FROM gold.dim_customers LIMIT 10;

In [0]:
-- VIEW 2: Product Dimension
CREATE OR REPLACE VIEW dim_products AS
SELECT
  ROW_NUMBER () OVER (ORDER BY pn.prd_start_dt, pn.prd_key) AS product_key,
  pn.prd_id AS product_id,
  pn.prd_key AS product_number,
  pn.prd_nm AS product_name,
  pn.cat_id AS category_id,
  pc.cat AS category,
  pc.subcat AS subcategory,
  pc.maintenance,
  pn.prd_cost AS cost,
  pn.prd_line AS product_line,
  pn.prd_start_dt AS start_date
FROM silver_crm.silver_crm_prd_info pn
LEFT JOIN silver_erp.silver_erp_px_cat_g1v2 pc
  ON TRIM(UPPER(pn.cat_id)) = TRIM(UPPER(pc.id))
WHERE pn.prd_end_dt IS NULL;

In [0]:
-- VIEW 3: Sales Fact Table
CREATE OR REPLACE VIEW fact_sales AS
SELECT
  sd.sls_ord_num AS order_number,
  pr.product_key,
  cu.customer_key,
  sd.sls_order_dt AS order_date,
  sd.sls_ship_dt AS shipping_date,
  sd.sls_due_dt AS due_date,
  sd.sls_sales AS sales_amount,
  sd.sls_quantity AS quantity,
  sd.sls_price AS price
FROM silver_crm.silver_crm_sales_details sd
LEFT JOIN dim_products pr
  ON TRIM(UPPER(sd.sls_prd_key)) = TRIM(UPPER(pr.product_number))
LEFT JOIN dim_customers cu
  ON TRIM(UPPER(sd.sls_cust_id)) = TRIM(UPPER(cu.customer_number));

In [0]:
DESCRIBE TABLE silver_crm.silver_crm_sales_details;

In [0]:
SELECT * FROM fact_sales LIMIT 10;

In [0]:
-- VALIDATION: Join check
SELECT *
FROM fact_sales f
LEFT JOIN dim_customers c ON f.customer_key = c.customer_key
LEFT JOIN dim_products p ON f.product_key = p.product_key
WHERE p.product_key IS NULL;

In [0]:
SELECT *
FROM fact_sales f
LEFT JOIN dim_customers c ON f.customer_key = c.customer_key
WHERE c.customer_key IS NULL;